In [49]:
import pandas as pd

dados = pd.read_csv("machine-learning-carros-simulacao.csv").drop(columns=["Unnamed: 0"], axis=1)
dados.head()

,preco,vendido,idade_do_modelo,km_por_ano
0,30941.02,1,18,35085.22134
1,40557.96,1,20,12622.05362
2,89627.50,0,12,11440.79806
3,95276.14,0,3,43167.32682
4,117384.68,1,4,12770.11290


In [50]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score

x = dados[["preco", "idade_do_modelo", "km_por_ano"]]
y = dados["vendido"]

SEED = 5
np.random.seed(SEED)
treino_x, teste_x, treino_y, teste_y = train_test_split(x, y, test_size=0.25, stratify=y)
print("Treinaremos com %d elementos e testaremos com %d elementos" % (len(treino_x), len(teste_x)))

Treinaremos com 7500 elementos e testaremos com 2500 elementos


In [51]:
from sklearn.dummy import DummyClassifier

dummy_stratified = DummyClassifier(strategy='stratified')
dummy_stratified.fit(treino_x, treino_y)
acuracia = dummy_stratified.score(teste_x, teste_y) * 100

print("A acurácia do dummy stratified foi de %.2f%%" % acuracia)

A acurácia do dummy stratified foi de 51.00%


In [52]:
from sklearn.tree import DecisionTreeClassifier

SEED = 5
np.random.seed(SEED)
modelo = DecisionTreeClassifier(max_depth=2)
modelo.fit(treino_x, treino_y)
previsoes = modelo.predict(teste_x)

acuracia = accuracy_score(teste_y, previsoes) * 100
print ("A acurácia foi %.2f%%" % acuracia)

A acurácia foi 76.84%


In [53]:
from sklearn.model_selection import cross_validate

SEED = 65657587
np.random.seed(SEED)

modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo, x, y, cv = 3, return_train_score=False)
media = results['test_score'].mean()
desvio_padrao = results['test_score'].std()
print("Accuracy com cross validation, 3 [%.2f, %.2f]" % ((media - 2 *desvio_padrao) * 100, (media + 2 * desvio_padrao) * 100))

Accuracy com cross validation, 3 [74.99, 76.57]


In [54]:
SEED = 65657587
np.random.seed(SEED)

modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo, x, y, cv = 10, return_train_score=False)
media = results['test_score'].mean()
desvio_padrao = results['test_score'].std()
print("Accuracy com cross validation, 10 [%.2f, %.2f]" % ((media - 2 *desvio_padrao) * 100, (media + 2 * desvio_padrao) * 100))

Accuracy com cross validation, 10 [74.24, 77.32]


In [55]:
from sklearn.model_selection import KFold

def imprime_result(results):
    media = results['test_score'].mean()
    desvio_padrao = results['test_score'].std()
    print("Accuracy médio: %.2f" % (media * 100))
    print("Accuracy com cross validation = [%.2f, %.2f]" % ((media - 2 * desvio_padrao) * 100, (media + 2 * desvio_padrao) * 100))

In [56]:
cv = KFold(n_splits=10)

modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo, x, y, cv=cv, return_train_score=False)
imprime_result(results)

Accuracy médio: 75.78
Accuracy com cross validation = [74.37, 77.19]


In [57]:
cv = KFold(n_splits=10, shuffle=True)

modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo, x, y, cv=cv, return_train_score=False)
imprime_result(results)

Accuracy médio: 75.78
Accuracy com cross validation = [73.84, 77.72]


#### Simular situação horrível de azar

#### Pode ser uma situação de "azar" ou uma proporção de exemplos desbalanceado entre as classes.

In [58]:
dados_azar = dados.sort_values("vendido", ascending=True)
x_azar = dados_azar[["preco", "idade_do_modelo", "km_por_ano"]]
y_azar = dados_azar["vendido"]
dados_azar.head()

,preco,vendido,idade_do_modelo,km_por_ano
4999,74023.29,0,12,24812.80412
5322,84843.49,0,13,23095.63834
5319,83100.27,0,19,36240.72746
5316,87932.13,0,16,32249.56426
5315,77937.01,0,15,28414.50704


In [59]:
cv = KFold(n_splits=10)

modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo, x_azar, y_azar, cv=cv, return_train_score=False)
imprime_result(results)

Accuracy médio: 57.84
Accuracy com cross validation = [34.29, 81.39]


In [60]:
cv = KFold(n_splits=10, shuffle=True)

modelo = DecisionTreeClassifier(max_depth=2,)
results = cross_validate(modelo, x_azar, y_azar, cv=cv, return_train_score=False)
imprime_result(results)

Accuracy médio: 75.78
Accuracy com cross validation = [72.40, 79.16]


In [61]:
from sklearn.model_selection import StratifiedKFold

cv = StratifiedKFold(n_splits=10, shuffle=True)

modelo = DecisionTreeClassifier(max_depth=2,)
results = cross_validate(modelo, x_azar, y_azar, cv=cv, return_train_score=False)
imprime_result(results)

Accuracy médio: 75.78
Accuracy com cross validation = [73.00, 78.56]


In [62]:
np.random.seed(SEED)
dados["modelo"] = dados.idade_do_modelo + np.random.randint(-2, 3, size=10000)
dados.head()

,preco,vendido,idade_do_modelo,km_por_ano,modelo
0,30941.02,1,18,35085.22134,18
1,40557.96,1,20,12622.05362,18
2,89627.50,0,12,11440.79806,14
3,95276.14,0,3,43167.32682,4
4,117384.68,1,4,12770.11290,3


In [63]:
dados.modelo = dados.modelo + abs(dados.modelo.min()) + 1
dados.head()

,preco,vendido,idade_do_modelo,km_por_ano,modelo
0,30941.02,1,18,35085.22134,20
1,40557.96,1,20,12622.05362,20
2,89627.50,0,12,11440.79806,16
3,95276.14,0,3,43167.32682,6
4,117384.68,1,4,12770.11290,5


In [66]:
from sklearn.model_selection import GroupKFold

cv = GroupKFold(n_splits=10)

modelo = DecisionTreeClassifier(max_depth=2,)
results = cross_validate(modelo, x_azar, y_azar, cv=cv, return_train_score=False, groups=dados.modelo)
imprime_result(results)

Accuracy médio: 75.76
Accuracy com cross validation = [73.27, 78.24]
